In [1]:
import netCDF4 as nc
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cmocean as cmo
import os,sys,fnmatch,time
import numpy as np
import matplotlib.gridspec as gridspec
from matplotlib import animation
from salishsea_tools.nc_tools import scDataset
from matplotlib import colors
from matplotlib import ticker
from matplotlib.colors import LinearSegmentedColormap
import gsw

%matplotlib inline

import sys
sys.path.append('/ocean/imachuca/Canyons/analysis_mackenzie_canyon/notebooks/general_circulation')
import general_functions

import warnings
warnings.filterwarnings("ignore")

import time
from datetime import datetime 

/home/imachuca/anaconda3/envs/gsw/lib/python3.6/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


In [2]:
def get_vars_pressure(dirname, fname, meshmaskname, time_ind):
    
    filesU = general_functions.get_files(dirname, fname, 'grid_U') 
    filesT = general_functions.get_files(dirname, fname, 'grid_T')
    
    y,x = slice(1,-1,None), slice(1,-1,None)

    with scDataset(filesU) as dsU, scDataset(filesT) as dsT:
        vosaline0 = dsT.variables['vosaline'][time_ind, :, y, x]
        votemper0 = dsT.variables['votemper'][time_ind, :, y, x]
        sossheig0 = dsT.variables['sossheig'][time_ind, y, x]

    with nc.Dataset(os.path.join(dirname, meshmaskname), 'r') as dsM:
        tmask = dsM.variables['tmask'][0, :, y, x]
        tmask_ssh = dsM.variables['tmask'][0, 0, y, x]

    vosaline = np.ma.array(vosaline0, mask=1 - tmask)
    votemper = np.ma.array(votemper0, mask=1 - tmask)
    sossheig = np.ma.array(sossheig0, mask=1 - tmask_ssh)
    
    return vosaline, votemper, sossheig

In [3]:
kind = 'ideal'
case = 'base' 
dirname = '/ocean/imachuca/Canyons/results_mackenzie/extended_domain/'+kind+'_'+case+'/'
fname = '1_MCKNZ_1h_20170101_201701*'
meshmaskname = '1_mesh_mask.nc'

In [4]:
dep_ind = 9
vosaline, votemper, sossheig = get_vars_pressure(dirname, fname, meshmaskname, dep_ind)

In [5]:
vosaline.shape, votemper.shape, sossheig.shape

((80, 242, 272), (80, 242, 272), (242, 272))

In [6]:
# where did the depth dimension go?
rho = gsw.rho(CT=votemper, SA=vosaline, p=10.1325)
rho.shape

(242, 272)

In [7]:
rho = gsw.rho(CT=votemper[9, :, :], SA=vosaline[9, :, :], p=10.1325)
rho.shape

(272,)

In [8]:
rho = gsw.rho(CT=votemper, SA=vosaline)

TypeError: rho() missing 1 required positional argument: 'p'